## Day 3
Url: https://adventofcode.com/2022/day/3

Problem: each line of the input (`'./inputs/day3.txt'`) contains the inventory of items in a elf's backpack. The first half of the line is in one compartment of the backpack, the second half is in a second compartment. The elf who packed the items did a mistake and packed one item in both compartments for every elf. 

We need to find:
1) which was the misplaced element for each elf, and
2) assign to each misplaced element a score (given by the problem)
3) calculate the total score of all the misplaced elements

Scoring:
a to z: 1 to 26
A to Z: 27 to 52
